# Logistic regression submission

## Notebook set up

### Imports

In [ ]:
import pandas as pd

### Run configuration

In [ ]:
# Flag to control file paths for Kaggle vs other environments
KAGGLE = False

### Data loading

In [ ]:
# Set file paths based on environment
if KAGGLE:

    # Kaggle paths - data is in /kaggle/input/
    test_df_path = '/kaggle/input/playground-series-s5e12/test.csv'

else:
    # Otherwise, load data from course GitHub repository
    test_df_path = 'https://gperdrizet.github.io/FSA_devops/assets/data/unit3/diabetes_prediction_test.csv'

# Load the testing dataset
test_df = pd.read_csv(test_df_path)

# Display first few rows of training data
test_df.head()